## Convert PGN to extended LAN (xLAN)

In [13]:
def pgn_to_xlan():
    from src.data_preprocessing.pgn_to_xlan import pgn_to_xlan

    # TODO change to valid paths
    pgn_path = "/Users/cyrilgabriele/Documents/School/00_Courses/03_MLOPS/04_Project/ChessOps/data/pgn/Carlsen.pgn"
    lan_path = "/Users/cyrilgabriele/Documents/School/00_Courses/03_MLOPS/04_Project/ChessOps/data/xlan/carlsen.xlanplus"

    min_number_of_moves_per_game = 0
    number_of_games_to_write = -1  # -1 for all games

    pgn_to_lan = pgn_to_xlan(
        pgn_path,
        lan_path,
        min_number_of_moves_per_game=min_number_of_moves_per_game,
        number_of_games_to_write=number_of_games_to_write,
        generate_all_moves=False,
        log=False,
        xLanPlus=True,
    )

    pgn_to_lan.convert_pgn_parallel()

pgn_to_xlan()

## Check Common and Duplicate Lines


In [14]:
"""
Use check_duplicates_and_common_lines to check if there are duplicates or common lines in two files.
"""

def check_duplicates():
    from src.data_preprocessing.check_duplicates_and_common_lines import (
        check_duplicates_and_common_lines,
    )   
    
    # TODO what is the validation file?
    training_file = "/Users/cyrilgabriele/Documents/School/00_Courses/03_MLOPS/04_Project/ChessOps/data/xlan/carlsen.xlanplus"
    validation_file = (
        "/Users/cyrilgabriele/Documents/School/00_Courses/03_MLOPS/04_Project/ChessOps/data/xlan/carlsen.xlanplus"
    )

    check_duplicates_and_common_lines(training_file, validation_file, delete_common=False, delete_duplicates_from_file_1=True, delete_duplicates_from_file_2=False)

## Tokenize Data


In [9]:
def tokenize_data():
    from src.tokenizer.tokenizer import tokenize_file
    
    xLAN_path = "/Users/cyrilgabriele/Documents/School/00_Courses/03_MLOPS/04_Project/ChessOps/data/xlan/carlsen.xlanplus" # Inout Path 
    token_path = "./src/tokenizer/xlanplus_tokens.json" # keep this, correct like this
    tokenized_path = "/Users/cyrilgabriele/Documents/School/00_Courses/03_MLOPS/04_Project/ChessOps/data/tokens/carlsen.tok" # Output path

    tokenize_file(token_path, xLAN_path, tokenized_path, batch_size=20000) # eventually smaller batch size

## Remove lines with more than x tokens


In [10]:
def remove_lines_with_too_many_tokens(input_file_path, output_file_path, token_limit=768):
    with open(input_file_path, "r") as file:
        lines = file.readlines()

    print(f"Number of lines in {input_file_path}: {len(lines)}")
    lines_to_keep = []
    removed_count = 0

    for line in lines:
        if len(line.split()) <= token_limit:
            lines_to_keep.append(line)
        else:
            removed_count += 1

    print(f"Number of lines in {output_file_path}: {len(lines_to_keep)}")
    with open(output_file_path, "w") as file:
        file.writelines(lines_to_keep)

    return removed_count

In [11]:
def remove(): 
    input_file_path = "/Users/cyrilgabriele/Documents/School/00_Courses/03_MLOPS/04_Project/ChessOps/data/tokens/carlsen.tok"
    output_file_path = "/Users/cyrilgabriele/Documents/School/00_Courses/03_MLOPS/04_Project/ChessOps/data/tokens/carlsen_max_768.tok"
    removed_lines = remove_lines_with_too_many_tokens(input_file_path, output_file_path)
    print(f"Number of removed lines: {removed_lines}")